## Things you need
+ config file with api key
+ Your user agent info can get https://www.whatismybrowser.com/detect/what-is-my-user-agent and replace the headers variable with your info
    + header variable is line 10 in both scraping block
+ Script splits up your sources into 2 queries and saves them as csv files.  I created a csv folder for them

## change query
Hit other candidates as well and make sure that the range (20,31) is no more than 30 days ago.  If you get a out of range error just change 20 to whatever day of month it is, or day of month +1

In [1]:
import pandas as pd
import config
import math
import csv
import sys
from newsapi import NewsApiClient
import newspaper
import requests
from newspaper import fulltext
import time
# Hit Api with credentials
newsapi = NewsApiClient(api_key=config.api_key)


## global variables
candidates_list = []
total_count = []


def error_checks():
    ##validate that df shape[0] is equal to expected query count
    print("we should expect: {} articles".format(sum(total_count)))
    print("we have: {} articles".format(Cand_df.shape[0]))


    ## test 100 results theory
    over_100 = sum([x-100 for x in total_count if x>100 ])
    print("{} results came into query that exceeded 100 hits in a day.".format(over_100))


    ##unique links
    unique_array = Cand_df.url.unique()
    print("we have {} unique links".format(unique_array.shape[0]))
    
    duplicateRowsDF = Cand_df[Cand_df.duplicated("url")]
    print("we have duplciate shape df of: {} ".format(duplicateRowsDF.shape))

def clean_query(query):
    for x in query['articles']:
        try:
            x["source"] = x["source"]["name"]
        except:
            pass
        try:
            x['publishedAt'] = str.split(x['publishedAt'], "T")[0]
        except:
            pass
        try:
            del x['urlToImage']
        except KeyError:
            pass
    my_df = pd.DataFrame(query["articles"])
    return my_df

def hit_api(start, end, q, myStr):

    # catch bug with formatted strings for dates
    if end < 10:
        start_str = "0" + str(start)
        end_str = "0" + str(end)
    elif end == 10:
        start_str = "0" + str(start)
        end_str = str(end)
    else:
        start_str = str(start)
        end_str = str(end)

    # API query
    all_articles = newsapi.get_everything(q=q,
                                          sources=myStr,
                                          language='en',
                                          from_param='2019-10-{}'.format(
                                              start_str),
                                          to='2019-10-{}'.format(end_str),
                                          sort_by='relevancy',
                                          page_size=100,
                                          page=1)

    # get count
    total_pages = math.ceil(all_articles["totalResults"]/100)
    print("query will return: " + str(all_articles["totalResults"]))

    # store count to check versus dimension of df later
    total_count.append(all_articles["totalResults"])

    # Clen query
    all_articles = clean_query(all_articles)

    # append to list
    candidates_list.append(all_articles)
    return(candidates_list)


In [2]:

## import data dictionary form github
url = 'https://raw.githubusercontent.com/mburke65/Data_698_Final/master/lists.csv'
df = pd.read_csv(url, error_bad_lines=False)

# picking data sources from file on github
a= df.iloc[[16,17,24],:]
b= df.iloc[[40,43,77],:]
list_sources = a["sources"].tolist()

# build out string for query request
myString = ",".join(list_sources)

list_sources2 = b["sources"].tolist()
myString2 = ",".join(list_sources2)



## Scrape Wall street journal and NYT

In [3]:

for x in range(1, 29):
    df = hit_api(x, x+1, 'Biden OR Joe Biden', myString2)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}
## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 88
query will return: 97
query will return: 117
query will return: 85
query will return: 31
query will return: 50
query will return: 64
query will return: 65
query will return: 73
query will return: 76
query will return: 49
query will return: 20
query will return: 35
query will return: 75
query will return: 99
query will return: 76
query will return: 47
query will return: 31
query will return: 18
query will return: 26
query will return: 38
query will return: 63
query will return: 75
query will return: 55
query will return: 31
query will return: 12
query will return: 21
query will return: 34
we should expect: 1551 articles
we have: 1534 articles
17 results came into query that exceeded 100 hits in a day.
we have 804 unique links
we have duplciate shape df of: (730, 7) 
1533
1532
1531
1530
1529
1528
1527
1526
1525
no words found
1524
1523
1522
no words found
1521
1520
1519
1518
1517
1516
1515
1514
1513
1512
1511
1510
1509
1508
1507
1506
1505
1504
1503
1502
1501
1500
14

64
63
62
61
60
59
58
57
56
55
54
53
52
51
50
49
48
47
46
45
no words found
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
no words found
28
27
26
25
24
23
22
21
20
no words found
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [4]:
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

## Scrape WAPO New York magazine


In [5]:
Cand_df=None

for x in range(1, 29):
    df = hit_api(x, x+1, 'Biden OR Joe Biden', myString)

# collapse list on itself to build big df
Cand_df = pd.concat(df)
Cand_df = Cand_df.reset_index(drop=True)
error_checks()


## Build counts as check on progress of loop. will count down Url left to hit
counts=Cand_df['url'].shape[0]
list_full_text=[]
for link in Cand_df['url']:
    counts-=1
    print(counts)
    html = requests.get(link,headers=headers).text
    try: text = fulltext(html)
    except: 
        print("no words found")
        text="no words found"
        list_full_text.append(text)
        continue
    list_full_text.append(text)
Cand_df['full_art']=pd.Series(list_full_text)
Cand_df.to_csv('data_csvs/data1'+str(time.time())+'.csv')

query will return: 99
query will return: 119
query will return: 118
query will return: 76
query will return: 42
query will return: 47
query will return: 56
query will return: 67
query will return: 80
query will return: 85
query will return: 53
query will return: 34
query will return: 48
query will return: 70
query will return: 113
query will return: 103
query will return: 56
query will return: 31
query will return: 15
query will return: 18
query will return: 28
query will return: 50
query will return: 63
query will return: 60
query will return: 44
query will return: 27
query will return: 28
query will return: 39
we should expect: 3220 articles
we have: 3150 articles
70 results came into query that exceeded 100 hits in a day.
we have 1649 unique links
we have duplciate shape df of: (1501, 7) 
3149
3148
3147
3146
3145
3144
3143
3142
3141
no words found
3140
3139
3138
no words found
3137
3136
3135
3134
3133
3132
3131
3130
3129
3128
3127
3126
3125
3124
3123
3122
3121
3120
3119
3118
3117
31

1854
1853
1852
1851
1850
1849
1848
1847
1846
1845
1844
1843
1842
1841
1840
1839
1838
1837
1836
1835
1834
1833
1832
1831
1830
1829
1828
no words found
1827
1826
1825
1824
1823
1822
1821
1820
1819
1818
no words found
1817
1816
1815
1814
1813
1812
1811
1810
1809
1808
1807
1806
1805
1804
1803
1802
1801
1800
1799
1798
1797
1796
1795
1794
1793
1792
1791
1790
1789
1788
1787
1786
1785
1784
1783
1782
1781
1780
1779
1778
1777
1776
1775
1774
1773
1772
1771
1770
1769
1768
1767
1766
1765
1764
1763
1762
1761
no words found
1760
1759
1758
1757
1756
1755
1754
1753
no words found
1752
1751
1750
no words found
1749
1748
1747
1746
1745
1744
1743
1742
1741
1740
1739
1738
1737
1736
1735
1734
1733
1732
1731
1730
1729
1728
1727
1726
1725
1724
1723
1722
1721
1720
1719
1718
1717
1716
1715
1714
1713
1712
1711
1710
no words found
1709
1708
1707
1706
no words found
1705
1704
1703
1702
1701
1700
1699
1698
1697
1696
1695
1694
1693
1692
1691
1690
1689
1688
1687
1686
1685
1684
1683
1682
1681
1680
1679
1678
1677
1676


43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [6]:
my_df=pd.read_csv('Vikas.csv')

In [7]:
my_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'author', 'content', 'description',
       'publishedAt', 'source', 'title', 'url', 'full_art'],
      dtype='object')

In [8]:
my_df["full_art"]

0       “Young people are constantly signaling what’s ...
1       WASHINGTON — For months this spring and summer...
2       WASHINGTON — President Trump pressed the Ukrai...
3       WASHINGTON — President Trump on Saturday dismi...
4       Though he has yet to call for impeachment proc...
5       Why is this coming up now?\r\r\n\r\r\nAn intel...
6       CEDAR RAPIDS, Iowa — Former Vice President Jos...
7       “Some government lawyer would be nervous to do...
8       Will this fine-tuning be enough in an agitated...
9       President Trump in a July phone call repeatedl...
10      Kamala Harris’s campaign says she is focusing ...
11      President Trump pressed on with calls for an i...
12      WASHINGTON — Everyone here is keyed up for the...
13      To the Editor:\r\r\n\r\r\nRe “Claim on Trump I...
14      WASHINGTON—Elizabeth Warren is building a broa...
15      Presidents these days always seem to get worse...
16      On Thursday, Mr. Atkinson appeared before a me...
17      Mayor 

In [9]:
Cand_df['full_art']

0       Former VP Joe Biden, at the 2020 Gun Forum in ...
1       Good morning and welcome to Fox News First. He...
2       Rachel Maddow looks at how Paul Manafort used ...
3       Former VP Joe Biden, at the 2020 Gun Forum in ...
4       Secretary of State Mike Pompeo was on the July...
5       Activist and filmmaker Michael Moore joins MSN...
6       A new House subpoena for Rudy Giuliani could b...
7       Sen. John Kennedy (R-La.) issued a notable pre...
8                                          no words found
9       Today’s installment of campaign-related news i...
10      President Trump put a spotlight on a 2014 phot...
11                                         no words found
12      EXCLUSIVE: A photo obtained by Fox News' "Tuck...
13      The impeachment war between President Trump an...
14      Former Democratic Sen. Robert Torricelli told ...
15      Former Senator Robert Torricelli (D-NJ) served...
16      Then-Vice President Joe Biden’s 2013 official ...
17      The ca